In [ ]:
import numpy as np
import pickle
import mygene
import gzip

In [ ]:
#Homo_sapiens
ref='GRCh38'
path_to_ref='/home/vasilis/refs/Homo_sapiens.GRCh38.cdna.all.fa.gz'      
ens_='ENS'

# #Mus_musculus
# ref='Mus_musculus.GRCm38'
# path_to_ref='/home/vasilis/refs/Mus_musculus.GRCm38.cdna.all.fa.gz' 
# ens_='ENSMUS'

#### tr2g and g2tr

In [ ]:
def readENS_ids(path_to_ref):
    TX_to_ENSG={}
    ENSG_isoforms={}
    tx_cnt=0;
    with gzip.open(path_to_ref) as f:
        for line in f:
            if line.decode('UTF-8')[0]=='>':
                    liner=line.decode('UTF-8')
                    ensg=ens_+'G'+liner.split(ens_+'G',1)[1][:11]
                    enst=ens_+'T'+liner.split(ens_+'T',1)[1][:11]
                    TX_to_ENSG[enst] = ensg
                    ENSG_isoforms[ensg] = ENSG_isoforms.get(ensg, [])
                    ENSG_isoforms[ensg].append(enst)
                    tx_cnt+=1 
    return [TX_to_ENSG,ENSG_isoforms]

In [ ]:
[TX_to_ENSG,ENSG_isoforms]=readENS_ids(path_to_ref) 

In [ ]:
with open(ref+'_tr2g.pickle', 'wb') as handle:
    pickle.dump(TX_to_ENSG, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(ref+'_g2tr.pickle', 'wb') as handle:
    pickle.dump(ENSG_isoforms, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ENSGLIST=list(np.unique(list(TX_to_ENSG.values())))

In [ ]:
print('number of genes: ',len(ENSGLIST))

#### ginfo and g2n

In [ ]:
mg = mygene.MyGeneInfo()
ginfo = mg.querymany(ENSGLIST, scopes='ensembl.gene',returnall=True)

In [ ]:
g2n = {}
count_exept=0
for g in ginfo['out']:
    try:
        gene_id=str(g['query'])
        gene_name=str(g['symbol'])
        
        g2n[gene_id] = g2n.get(gene_id, [])
        g2n[gene_id].append(str(g['symbol']))                
    except KeyError:
        count_exept+=1
        g2n[ str(g['query']) ] = [str(g['query'])]
        
n2g = {}
count_exept=0
for g in ginfo['out']:
    try:
        gene_id=str(g['query'])
        gene_name=str(g['symbol'])
        
        n2g[gene_name] = n2g.get(gene_name, [])
        n2g[gene_name].append(gene_id)                
    except KeyError:
        count_exept+=1
        n2g[ gene_id ] = [gene_id]

In [ ]:
with open(ref+'_ginfo.pickle', 'wb') as handle:
    pickle.dump(ginfo, handle, protocol=pickle.HIGHEST_PROTOCOL)   
    
with open(ref+'_g2n.pickle', 'wb') as handle:
    pickle.dump(g2n, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    
with open(ref+'_n2g.pickle', 'wb') as handle:
    pickle.dump(n2g, handle, protocol=pickle.HIGHEST_PROTOCOL)  